<a href="https://colab.research.google.com/github/MeenatchiKV/My-ML-Learnings/blob/master/Date_Parser_and_Pearson_Correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Date Parser**

## **Parse Date Columns and Find the Difference between each of the date columns.**

Import pandas and datetime modules

In [ ]:
import pandas as pd
import datetime


Read any Dataset

Link to the HR_Dataset:
https://www.kaggle.com/rhuebner/human-resources-data-set

In [ ]:
df = pd.read_csv("HR_Dataset.csv")
df.head()

,Employee_Name,EmpID,MarriedID,MaritalStatusID,GenderID,EmpStatusID,DeptID,PerfScoreID,FromDiversityJobFairID,PayRate,Termd,PositionID,Position,State,Zip,DOB,Sex,MaritalDesc,CitizenDesc,HispanicLatino,RaceDesc,DateofHire,DateofTermination,TermReason,EmploymentStatus,Department,ManagerName,ManagerID,RecruitmentSource,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30
0,"Brown, Mia",1.103024e+09,1.0,1.0,0.0,1.0,1.0,3.0,1.0,28.50,0.0,1.0,Accountant I,MA,1450.0,11/24/87,F,Married,US Citizen,No,Black or African American,10/27/2008,NaN,N/A - still employed,Active,Admin Offices,Brandon R. LeBlanc,1.0,Diversity Job Fair,Fully Meets,2.04,2.0,6.0,1/15/2019,0.0
1,"LaRotonda, William",1.106027e+09,0.0,2.0,1.0,1.0,1.0,3.0,0.0,23.00,0.0,1.0,Accountant I,MA,1460.0,04/26/84,M,Divorced,US Citizen,No,Black or African American,1/6/2014,NaN,N/A - still employed,Active,Admin Offices,Brandon R. LeBlanc,1.0,Website Banner Ads,Fully Meets,5.00,4.0,4.0,1/17/2019,0.0
2,"Steans, Tyrone",1.302053e+09,0.0,0.0,1.0,1.0,1.0,3.0,0.0,29.00,0.0,1.0,Accountant I,MA,2703.0,09/01/86,M,Single,US Citizen,No,White,9/29/2014,NaN,N/A - still employed,Active,Admin Offices,Brandon R. LeBlanc,1.0,Internet Search,Fully Meets,3.90,5.0,5.0,1/18/2019,0.0
3,"Howard, Estelle",1.211051e+09,1.0,1.0,0.0,1.0,1.0,3.0,0.0,21.50,1.0,2.0,Administrative Assistant,MA,2170.0,09/16/85,F,Married,US Citizen,No,White,2/16/2015,04/15/15,NaN,Terminated for Cause,Admin Offices,Brandon R. LeBlanc,1.0,Pay Per Click - Google,Fully Meets,3.24,3.0,4.0,NaN,NaN
4,"Singh, Nan",1.307060e+09,0.0,0.0,0.0,1.0,1.0,3.0,0.0,16.56,0.0,2.0,Administrative Assistant,MA,2330.0,05/19/88,F,Single,US Citizen,No,White,5/1/2015,NaN,N/A - still employed,Active,Admin Offices,Brandon R. LeBlanc,1.0,Website Banner Ads,Fully Meets,5.00,3.0,5.0,1/15/2019,0.0


Defining the Date Column Parser

In [ ]:
import re
def parse_col(column):
  coll = df[column]
  match = re.match(r'(\d+/\d+/\d+)|(\d+ \w\w\w \d+)',str(coll[0]))
  # To Match
  # dd/mm/yy
  # dd/mm/yyyy
  # yyyy/mm/dd
  # dd mmm yyyy

  if match: 
    return True
  else: return False

Defining Datatype changer

Change Datatype from Object to datetime

In [ ]:
def change_type(col):
  df[col]=pd.to_datetime(df[col])
  return df[col].dtype

Finding the list of Date Columns

In [ ]:
date_cols = []
for col in df.columns:
   if df[col].dtype == 'datetime64[ns]':
     date_cols.append(col)
   elif df[col].dtype == 'object':
     is_date = parse_col(col)
     if is_date==True:
       change = change_type(col)
       if change != 'object':
         date_cols.append(col)

print(date_cols)

['DOB', 'DateofHire', 'LastPerformanceReview_Date']


Creating new columns and Filling the differences of dates

In [ ]:
n = len(date_cols)
# n columns
n_cols = []
diff_df = pd.DataFrame()
for i in range(n-1):
  for j in range(i+1,n):
    n_name = date_cols[i]+'-'+date_cols[j]
    diff_df[n_name] = df[date_cols[i]]-df[date_cols[j]]
    n_cols.append(n_name)
print(n_cols)

['DOB-DateofHire', 'DOB-LastPerformanceReview_Date', 'DateofHire-LastPerformanceReview_Date']


In [ ]:
diff_df[n_cols[0]].head()

0    -7643 days
1   -10847 days
2   -10255 days
3   -10745 days
4    -9843 days
Name: DOB-DateofHire, dtype: timedelta64[ns]

In [ ]:
diff_df.head()

,DOB-DateofHire,DOB-LastPerformanceReview_Date,DateofHire-LastPerformanceReview_Date
0,-7643 days,-11375 days,-3732 days
1,-10847 days,-12684 days,-1837 days
2,-10255 days,-11827 days,-1572 days
3,-10745 days,NaT,NaT
4,-9843 days,-11198 days,-1355 days




---



## **2. Pearson Correlation**

Drop Features having Pearson correlation more than 0.85

Import necessary modules

In [ ]:
from numpy import mean
from numpy import std
from numpy.random import seed
# seed random number generator
seed(1)
import pandas as pd


Load Dataset

Chosen Dataset: Iris Dataset

In [ ]:
path = "https://raw.githubusercontent.com/uiuc-cse/data-fa14/gh-pages/data/iris.csv"
df = pd.read_csv(path)
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


The following are the columns in the dataset

In [ ]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width',
       'species'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal_length  150 non-null    float64
 1   sepal_width   150 non-null    float64
 2   petal_length  150 non-null    float64
 3   petal_width   150 non-null    float64
 4   species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


Seperating features and columns

Not slicing the dataset. But just getting the column names

In [ ]:
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
classes = ['species']

Calculating the pearson coefficients for every pair of features.

The following function creates a dictionary called pearson_dict. The pearson_dict will hold information regarding pairs of variables that have pearason correlation of more than 0.85. pearson_dict will contain a key,value pair where the key is the variable name and value is a list in the form [count,[related_feature]]
pearson_dict =  {var:[count,[related_feature]]}

that is count implies the number of other attributes the 'var' is related to. and the following list contains those other attributes.

In [ ]:
from scipy.stats import pearsonr
pearson_dict = {}
def calc_pearson(features):
  #pearson_dict = {}
  n = len(features)
  for _ in range(n):
    pearson_dict[features[_]]=[0,[]]

  for i in range(n-1):
    for j in range(i+1,n):
      n_name=features[i]+'-'+features[j]
      corr,_ = pearsonr(df[features[i]],df[features[j]])
      if corr>0.85:
        pearson_dict[features[i]][0] = pearson_dict[features[i]][0] + 1
        pearson_dict[features[i]][1].append(features[j])
        pearson_dict[features[j]][0] = pearson_dict[features[j]][0] + 1
        pearson_dict[features[j]][1].append(features[i])

  print(pearson_dict)


calc_pearson(features)

{'sepal_length': [1, ['petal_length']], 'sepal_width': [0, []], 'petal_length': [2, ['sepal_length', 'petal_width']], 'petal_width': [1, ['petal_length']]}


{'sepal_length': [1, ['petal_length']], 'sepal_width': [0, []], 'petal_length': [2, ['sepal_length', 'petal_width']], 'petal_width': [1, ['petal_length']]}

The above dictionary is explained here:

sepal_length is related to 1 other variable - petal_length

sepal_width is not related to any variable

petal_length is related to 2 other variables - sepal_length and petal_width

petal_width is related to 1 other variable - petal_length


In [ ]:
max_val = -1
max_key = []
for k,v in pearson_dict.items():
  if v[0] > max_val:
    max_val = v[0]
    max_key = [k]
    continue
  if v[0] == max_val:
    max_key.append(k)

print(max_key)
print(max_val)

['petal_length']
2


The above code takes into account the maximum count value and the list of variables with that count value.

that is , max_val = max(count)

max_key = list of variable with count = max_val

In [ ]:
if max_val != 0:
  for m in max_key:
    maxx = pearson_dict[m]
    for _ in range(maxx[0]):
      pearson_dict[maxx[1][_]][0]= pearson_dict[maxx[1][_]][0] - 1
      pearson_dict[maxx[1][_]][1].remove(m)
    del pearson_dict[m]
    features.remove(m)

    print(pearson_dict)
    print(features)

{'sepal_length': [0, []], 'sepal_width': [0, []], 'petal_width': [0, []]}
['sepal_length', 'sepal_width', 'petal_width']


Now the above code deletes the max_key variables from the pearson_dict and the rest are the necessary unrelated features.

---



## Executing Pearson Correlation to Breast Cancer wisconsin **dataset**

In [ ]:
path = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/breast-cancer-wisconsin.csv"
df = pd.read_csv(path,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


Add the column names to the dataset

['clump_thickness', 'unif_cell_size', 'unif_cell_shape',
       'marg_adhesion', 'single_apith_cell', 'bare_nuclei', 'bland_chrom',
       'norm_nucleoli', 'mitoses', 'class']

In [ ]:
df.columns = ['clump_thickness', 'unif_cell_size', 'unif_cell_shape', 'marg_adhesion',
              'single_apith_cell', 'bare_nuclei', 'bland_chrom',
       'norm_nucleoli', 'mitoses', 'class']
df.head       


<bound method NDFrame.head of      clump_thickness  unif_cell_size  ...  mitoses  class
0                  5               1  ...        1      2
1                  5               4  ...        1      2
2                  3               1  ...        1      2
3                  6               8  ...        1      2
4                  4               1  ...        1      2
..               ...             ...  ...      ...    ...
694                3               1  ...        1      2
695                2               1  ...        1      2
696                5              10  ...        2      4
697                4               8  ...        1      4
698                4               8  ...        1      4

[699 rows x 10 columns]>

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   clump_thickness    699 non-null    int64 
 1   unif_cell_size     699 non-null    int64 
 2   unif_cell_shape    699 non-null    int64 
 3   marg_adhesion      699 non-null    int64 
 4   single_apith_cell  699 non-null    int64 
 5   bare_nuclei        699 non-null    object
 6   bland_chrom        699 non-null    int64 
 7   norm_nucleoli      699 non-null    int64 
 8   mitoses            699 non-null    int64 
 9   class              699 non-null    int64 
dtypes: int64(9), object(1)
memory usage: 54.7+ KB


In [ ]:
df.columns

Index(['clump_thickness', 'unif_cell_size', 'unif_cell_shape', 'marg_adhesion',
       'single_apith_cell', 'bare_nuclei', 'bland_chrom', 'norm_nucleoli',
       'mitoses', 'class'],
      dtype='object')

In [ ]:
df.replace('?',-99999,inplace=True)
df['bare_nuclei'] = df['bare_nuclei'].astype(int)
df.columns

Index(['clump_thickness', 'unif_cell_size', 'unif_cell_shape', 'marg_adhesion',
       'single_apith_cell', 'bare_nuclei', 'bland_chrom', 'norm_nucleoli',
       'mitoses', 'class'],
      dtype='object')

In [ ]:
features = ['clump_thickness', 'unif_cell_size', 'unif_cell_shape',
       'marg_adhesion', 'single_apith_cell', 'bare_nuclei', 'bland_chrom',
       'norm_nucleoli', 'mitoses']
classes = ['class']

df = df.dropna()

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 699 entries, 0 to 698
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   clump_thickness    699 non-null    int64
 1   unif_cell_size     699 non-null    int64
 2   unif_cell_shape    699 non-null    int64
 3   marg_adhesion      699 non-null    int64
 4   single_apith_cell  699 non-null    int64
 5   bare_nuclei        699 non-null    int64
 6   bland_chrom        699 non-null    int64
 7   norm_nucleoli      699 non-null    int64
 8   mitoses            699 non-null    int64
 9   class              699 non-null    int64
dtypes: int64(10)
memory usage: 60.1 KB


In [ ]:
from scipy.stats import pearsonr
pearson_dict = {}
def calc_pearson(features):
  #pearson_dict = {}
  n = len(features)
  for _ in range(n):
    pearson_dict[features[_]]=[0,[]]

  for i in range(n-1):
    for j in range(i+1,n):
      n_name=features[i]+'-'+features[j]
      corr,_ = pearsonr(df[features[i]],df[features[j]])
      if corr>0.85:
        pearson_dict[features[i]][0] = pearson_dict[features[i]][0] + 1
        pearson_dict[features[i]][1].append(features[j])
        pearson_dict[features[j]][0] = pearson_dict[features[j]][0] + 1
        pearson_dict[features[j]][1].append(features[i])

  print(pearson_dict)


calc_pearson(features)

{'clump_thickness': [0, []], 'unif_cell_size': [1, ['unif_cell_shape']], 'unif_cell_shape': [1, ['unif_cell_size']], 'marg_adhesion': [0, []], 'single_apith_cell': [0, []], 'bare_nuclei': [0, []], 'bland_chrom': [0, []], 'norm_nucleoli': [0, []], 'mitoses': [0, []]}


In [ ]:
max_val = -1
max_key = []
for k,v in pearson_dict.items():
  if v[0] > max_val:
    max_val = v[0]
    max_key = [k]
    continue
  if v[0] == max_val:
    max_key.append(k)

print(max_key)
print(max_val)

['unif_cell_size', 'unif_cell_shape']
1


Execute this only when atleast one pair is correlated.

In [ ]:
if max_val != 0:
  for m in max_key:
    maxx = pearson_dict[m]
    for _ in range(maxx[0]):
      pearson_dict[maxx[1][_]][0]= pearson_dict[maxx[1][_]][0] - 1
      pearson_dict[maxx[1][_]][1].remove(m)
    del pearson_dict[m]

    print(pearson_dict)

{'clump_thickness': [0, []], 'unif_cell_shape': [0, []], 'marg_adhesion': [0, []], 'single_apith_cell': [0, []], 'bare_nuclei': [0, []], 'bland_chrom': [0, []], 'norm_nucleoli': [0, []], 'mitoses': [0, []]}
{'clump_thickness': [0, []], 'marg_adhesion': [0, []], 'single_apith_cell': [0, []], 'bare_nuclei': [0, []], 'bland_chrom': [0, []], 'norm_nucleoli': [0, []], 'mitoses': [0, []]}
